# 스타벅스 매장 데이터와 지도


* json_normalize(): JSON 데이터 DataFrame으로 전환
* folium 을 사용하여 지도에 위치 표시

# 스타벅스 매장찾기
http://www.istarbucks.co.kr

* STORE >> 매장 찾기
* 지역검색 >> 서울 >> 전체

In [25]:
import requests
import json
import pandas as pd

In [26]:
data = {
    'ins_lat':'',     
    'ins_lng':'',
    'p_sido_cd':'01', # 01=서울시, 02=광주, 03=대구, 04:대전, 05=부산, 08=경기 ... 16=제주
    'p_gugun_cd':'',  # 세부지역 (지정하지 않으면 시/도 전체)
    'in_biz_cd':'',
    'set_date':'',
    'iend':'1000',
}
    
url = 'https://www.starbucks.co.kr/store/getStore.do'
r = requests.post(url, data=data)    # request.get()과 동일, params 대신 data 사용.

In [27]:
print(r.text[:1000]) # 수신된 데이터의 앞부분만 확인

{"list":[{"seq":0,"sido_cd":null,"sido_nm":null,"gugun_cd":null,"gugun_nm":null,"code_order":null,"view_yn":null,"store_num":null,"sido":null,"gugun":null,"address":null,"new_img_nm":null,"p_pro_seq":0,"p_view_yn":null,"p_sido_cd":"","p_gugun_cd":"","p_store_nm":null,"p_theme_cd":null,"p_wireless_yn":null,"p_smoking_yn":null,"p_book_yn":null,"p_music_yn":null,"p_terrace_yn":null,"p_table_yn":null,"p_takeout_yn":null,"p_parking_yn":null,"p_dollar_assent":null,"p_card_recharge":null,"p_subway_yn":null,"stb_store_file_renew":null,"stb_store_theme_renew":null,"stb_store_time_renew":null,"stb_store_lsm":null,"s_code":"1509","s_name":"역삼아레나빌딩","tel":"1522-3232","fax":"02-568-3763","sido_code":"01","sido_name":"서울","gugun_code":"0101","gugun_name":"강남구","addr":"서울특별시 강남구 역삼동 721-13 아레나빌딩","park_info":null,"new_state":null,"theme_state":"T05@T08@T16@T17@T20@T21@T30@@T52@P80@P90","new_bool":0,"search_text":"","ins_lat":"","ins_lng":"","in_distance":0,"out_distance":null,"all_search_cnt":-1,"add

데이터는 JSON으로 수신되며, 형태는 다음과 같다.

``` json    # fenced code block
{
  "list": [
      {
            // ... 중략 ...
            "s_code": "1311",
            "s_name": "방화DT",
            "tel": "02-2664-3480",
            "fax": "02-2664-3481",
            "sido_code": "01",
            "sido_name": "서울",
            "gugun_code": "0103",
            "gugun_name": "강서구",
            "addr": "서울특별시 강서구 방화동 293-4",
            "park_info": null,
            "new_state": null,
            "theme_state": "T17@T16@T09@T20@T01@T05@T08@T04",
            // ... 중략 ...
            "lat": "37.574339",
            "lot": "126.816415",
            "t22": 0
        },
        {
            // ... 중략 ...
            "s_code": "1267",
            "s_name": "마곡나루역",
            "tel": "02-3662-3504",
            "fax": "02-3662-3505",
            "sido_code": "01",
            "sido_name": "서울",
            "gugun_code": "0103",
            "gugun_name": "강서구",
            "addr": "서울특별시 강서구 마곡동 759-3 보타닉파크타워Ⅰ105,203,204호",
            "park_info": null,
            "new_state": null,
            "theme_state": "T08@T05@T04@T17@T16@P80@T20",
            // ... 중략 ...
            "lat": "37.56813",
            "lot": "126.82614",
            "t22": 0
        },
    ]
 }
```

# JSON to DataFrame
- json_normalize() 를 사용하여 JSON 데이터를 DataFrame로 전환 

In [28]:
jo = json.loads(r.text)

In [29]:
df = pd.json_normalize(jo, 'list')

In [30]:
df.head()

,seq,sido_cd,sido_nm,gugun_cd,gugun_nm,code_order,view_yn,store_num,sido,gugun,...,t22,t21,p90,t05,t30,t36,t27,t29,t43,t48
0,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
1,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
2,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
3,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0
4,0,None,None,None,None,None,None,None,None,None,...,0,0,0,0,0,0,0,0,0,0


In [31]:
df.shape

(590, 129)

In [32]:
# 컬럼수 129개
df.columns

Index(['seq', 'sido_cd', 'sido_nm', 'gugun_cd', 'gugun_nm', 'code_order',
       'view_yn', 'store_num', 'sido', 'gugun',
       ...
       't22', 't21', 'p90', 't05', 't30', 't36', 't27', 't29', 't43', 't48'],
      dtype='object', length=129)

In [33]:
# 주요한 컬럼 몇 가지 선택
               
df = df[['s_name', 'lat', 'lot', 'sido_name', 'gugun_name', 'doro_address', 'tel']]
df.head(10)

,s_name,lat,lot,sido_name,gugun_name,doro_address,tel
0,역삼아레나빌딩,37.501087,127.043069,서울,강남구,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,서울,강남구,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.5139309,127.0206057,서울,강남구,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,서울,강남구,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,대치재경빌딩R,37.494668,127.062583,서울,강남구,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232
5,봉은사역,37.515000,127.063196,서울,강남구,서울특별시 강남구 봉은사로 619 (삼성동),1522-3232
6,압구정윤성빌딩,37.5227934,127.0286009,서울,강남구,서울특별시 강남구 논현로 834 (신사동),1522-3232
7,코엑스별마당,37.510150,127.060275,서울,강남구,서울특별시 강남구 영동대로 513 (삼성동),1522-3232
8,삼성역섬유센터R,37.507750,127.060651,서울,강남구,서울특별시 강남구 테헤란로 518 (대치동),1522-3232
9,압구정R,37.5273669,127.033061,서울,강남구,서울특별시 강남구 언주로 861 (신사동),1522-3232


# 위도 경도 데이터 타입 변환

In [34]:
df.dtypes

s_name          object
lat             object
lot             object
sido_name       object
gugun_name      object
doro_address    object
tel             object
dtype: object

### lat, lot타입으로 전환 (str→float) 

In [36]:
df['lat'] = df['lat'].astype(float)
df['lot'] = df['lot'].astype(float)

In [37]:
df.dtypes

s_name           object
lat             float64
lot             float64
sido_name        object
gugun_name       object
doro_address     object
tel              object
dtype: object

In [38]:
df.head(5)

,s_name,lat,lot,sido_name,gugun_name,doro_address,tel
0,역삼아레나빌딩,37.501087,127.043069,서울,강남구,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,37.510178,127.022223,서울,강남구,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,37.513931,127.020606,서울,강남구,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,37.499517,127.031495,서울,강남구,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,대치재경빌딩R,37.494668,127.062583,서울,강남구,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232


In [39]:
(df.s_name == '시청').sum()

1

### 특정 지점

In [40]:
df[df.s_name == "광운대"] 

,s_name,lat,lot,sido_name,gugun_name,doro_address,tel
172,광운대,37.623148,127.060701,서울,노원구,서울특별시 노원구 석계로 104 (월계동),1522-3232


In [41]:
df[df['s_name'] == '시청']

,s_name,lat,lot,sido_name,gugun_name,doro_address,tel
495,시청,37.56629,126.979808,서울,중구,"서울특별시 중구 을지로 19, 삼성화재삼성빌딩 1층 (을지로1가)",1522-3232


In [42]:
df[df['s_name'] == '시청'][['lat','lot']]

,lat,lot
495,37.56629,126.979808


스타벅스 시청점의 위도(lat)와 경도(lot)는 각각  37.56629, 126.979808

# folium
https://github.com/python-visualization/folium

파이썬 지리정보 시각화 모듈  (문서: https://folium.readthedocs.io )

In [43]:
!pip install folium     

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
import folium         # python 지리정보 시각화 모듈

seoul_cityhall = (37.56629, 126.979808)

In [45]:
map_osm = folium.Map(location=seoul_cityhall)
# map_osm = folium.Map(location = kb)
map_osm

In [46]:
map_osm = folium.Map(location = seoul_cityhall, zoom_start=17)
map_osm

In [47]:
map_osm = folium.Map(location = seoul_cityhall, zoom_start=17, tiles='Stamen Toner')
map_osm

In [48]:
map_osm = folium.Map(location=seoul_cityhall, zoom_start=17)
folium.Marker(seoul_cityhall, popup='서울시청').add_to(map_osm)
folium.CircleMarker(seoul_cityhall, radius=50).add_to(map_osm)
map_osm

In [49]:
# 전체 지점
map_osm = folium.Map(location=seoul_cityhall, zoom_start=11)

for ix, row in df.iterrows():        # Iterate over DataFrame rows as (index, Series) pair
    location = (row['lat'], row['lot'])
    folium.Marker(location, 
                  popup=row['s_name']).add_to(map_osm)

map_osm

# 요약
* requests.post() 데이터 가져오기
* json_normalize(): JSON 데이터 DataFrame으로 전환
* folium 을 사용하여 지도에 위치 표시